<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/2_rag_bm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruções

* Estratégia 1
    * Usar segmentação de sentenças
    * BM25 como buscador

# Instalando pacotes

In [ ]:
!pip install -q llama-index llama-index-llms-groq llama-index-retrievers-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00


# Importando pacotes

In [ ]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
import json
import time
import random
from tqdm import tqdm
from rich.pretty import pprint

from llama_index.core import Document
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.groq import Groq
from llama_index.core.response.notebook_utils import display_source_node, display_response
from llama_index.core.evaluation import FaithfulnessEvaluator, CorrectnessEvaluator, RelevancyEvaluator

import nest_asyncio
nest_asyncio.apply()

from IPython.display import Markdown, display
import pandas as pd

# GitHub

In [ ]:
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

In [ ]:
DATA_DIR = '/content/ia024-projeto-rag'

In [ ]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [ ]:
def update_local_repo():
    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

## Criando documentos e nós

In [ ]:
import glob

def get_json_files(directory):
    return glob.glob(os.path.join(directory, "*.json"))


base_docs     = []

print(f"Carregando Estatuto e Organizações Didáticas...")
directory_path = f"{DATA_DIR}/2-dataset/"
json_files     = get_json_files(directory_path)

for json_file in json_files:
    print(f"Carregando: {json_file}")
    with open(json_file, 'r') as f:
        documento = json.load(f)

    documento_titulo = documento['titulo']
    documento_texto  = documento['texto_compacto']

    for parte in documento['partes']:
        metadata = {}
        metadata["documento"] = documento_titulo

        titulo_nome    = parte['titulo']
        titulo_artigos = parte['artigos']
        capitulos      = parte['capitulos']

        metadata["titulo"] = titulo_nome
        if titulo_artigos != "":
            doc = Document(text=titulo_artigos, metadata=metadata)
            base_docs.append(doc)

        for capitulo in capitulos:
            cap_titulo  = capitulo['titulo']
            cap_artigos = capitulo['artigos']

            metadata["capitulo"] = cap_titulo
            doc = Document(text=cap_artigos, metadata=metadata)
            base_docs.append(doc)
    print(f"{len(base_docs)} documentos carregados")

print(f"Carregando portarias: {directory_path}")
directory_path = f"{DATA_DIR}/2-dataset/2-boituva/portarias"
json_files     = get_json_files(directory_path)

for json_file in json_files:
    print(f"Carregando: {json_file}")
    with open(json_file, 'r') as f:
        documento = json.load(f)

    for portaria in documento:

        portaria_titulo = portaria['title:']
        portaria_texto  = portaria['texto_completo']

        metadata = {}
        metadata["campus"]    = portaria['campus']
        metadata["documento"] = portaria_titulo
        doc = Document(text=portaria_texto, metadata=metadata)
        base_docs.append(doc)

    print(f"{len(base_docs)} documentos carregados")

for i, doc in enumerate(base_docs):
    doc.id_ = f"doc_{i}"

print(f"Total de documentos carregados: {len(base_docs)}")

In [ ]:
pprint(docs[0])

Document(
│   id_='0',
│   embedding=None,
│   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO I - DA NATUREZA E DAS FINALIDADES. Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I. Reitoria, no endereço citado no §1º. II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista. VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano. XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva. XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré. XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga. XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro. XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba. XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. XXXI. Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila Nova Santana, Assis. § 3º- Para efeito da incidência das disposições qu

In [ ]:
# Esta classe separa o texto em sentenças e, de acordo com o parâmetro "window_size",
# cria janelas com a sentença atual mais 'window_size' sentenças antes e depois
# as janelas são salvas no metadata "window" e as sentenças em "original_sentence"
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=1, # com valor 1, as "janelas" terão tamanho 3: uma sentença antes, a sentença atual, uma sentença depois
    window_metadata_key="window",
    original_text_metadata_key="original_sentence",
)

nodes = node_parser.get_nodes_from_documents(docs)

In [ ]:
len(nodes)

264

In [ ]:
pprint(nodes[0])

TextNode(
│   id_='ad00c752-7898-4884-9a29-938b1e9a5369',
│   embedding=None,
│   metadata={
│   │   'window': 'ESTATUTO.  TÍTULO I - DA INSTITUIÇÃO.  CAPÍTULO I - DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. ',
│   │   'original_sentence': 'ESTATUTO. ',
│   │   'doc_name': 'Estatuto',
│   │   'category': 'Reitoria'
│   },
│   excluded_embed_metadata_keys=['window', 'original_sentence'],
│   excluded_llm_metadata_keys=['window', 'original_sentence'],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   node_id='0',
│   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   hash='c958262a924fc6e53db65cce9967b79c93c611568eb1a613ff6791791d3dfe21'
│   │   ),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
│   │   │   node_id='e51a19c2-2380-4781-9e71-41632ef2ff7a',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={
│   │   │   │   'window': 'ESTATUTO.  TÍTULO I - DA INSTITUIÇÃO.  CAPÍTULO I - DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº.  11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. ',
│   │   │   │   'original_sentence': 'TÍTULO I - DA INSTITUIÇÃO. '
│   │   │   },
│   │   │   hash='8f22f8fe4aba5ca0baa7c6c5cbd7ec7b57a0cf66c22597581eaff1c5e2d7c1d0'
│   │   )
│   },
│   text='ESTATUTO. ',
│   start_char_idx=0,
│   end_char_idx=10,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

# BM25

## Criando BM25

In [ ]:
# Criando o BM25
# com 'tokenizer=None', ele vai fazer a remoção de stop words e o stemming
retriever = BM25Retriever(nodes=nodes, tokenizer=None, similarity_top_k=10)

In [ ]:
# Testando a busca com BM25
retriever_nodes = retriever.retrieve("Qual o nome do IFSP?")
for node in retriever_nodes:
    display_source_node(node)

## Criando o 'motor de busca'

In [ ]:
# Este motor de busca será utilizado como tool pelo agente

# Após testes e análise do código, percebi que:
# - O BM25 faz a busca utilizando a sentença e não a janela
# - O resultado da busca é substituído pelas respectivas "janelas"
# - O LLM indicado é utilizado para "sintetizar" o conteúdo das janelas
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=Groq(model="llama3-70b-8192"),
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
)

In [ ]:
# Testando a busca
response = query_engine.query(
    "Qual o nome do IFSP?"
    #"Quais são os campi do IFSP?"
)
display_response(response)

**`Final Response:`** Não há menção explícita ao nome do IFSP no contexto fornecido.

In [ ]:
for node in response.source_nodes:
    display_source_node(node, source_length=500)

In [ ]:
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} " f"\n\n**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))


display_prompt_dict(query_engine.get_prompts())

# Avaliando

In [ ]:
def get_contexts(response):
    contexts = []
    for node in response.source_nodes:
        metadata = ', '.join(node.metadata.values())
        context  = node.get_content()
        contexts.append(f"Fonte: {metadata} # Contextos: {context}")
    return contexts

In [ ]:
faith_evaluator     = FaithfulnessEvaluator()
relevancy_evaluator = RelevancyEvaluator()
correct_evaluator   = CorrectnessEvaluator()
data = []

for pair in tqdm(pairs):
    result       = {}
    query        = pair['question']
    ground_truth = pair['answer']

    result['question']     = query
    result['ground_truth'] = ground_truth

    # Fazendo consulta
    response           = query_engine.query(query)
    result['answer']   = response.response
    result['contexts'] = get_contexts(response)


    # Avaliando Faithfulness
    eval_result                     = faith_evaluator.evaluate_response(response=response)
    result['faithfulness']          = eval_result.score
    result['faithfulness_feedback'] = eval_result.feedback

    # Avaliando Relevancy
    eval_result                  = relevancy_evaluator.evaluate_response(query=query, response=response)
    result['relevancy']          = eval_result.score
    result['relevancy_feedback'] = eval_result.feedback

    # Avaliando Correctness
    eval_result = correct_evaluator.evaluate(
        query=query,
        response=response.response,
        reference=ground_truth,
    )
    result['correctness']          = eval_result.score
    result['correctness_feedback'] = eval_result.feedback

    data.append(result)


df = pd.DataFrame(data)
df